# Explainable Boosting Machine (EBM) for Earned Value Management

# Model selection

## Dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Simulation  dataset
# Null model (5-rand) of comparison
data=pd.read_csv('./montecarlo/simulation_EV0.75_5-rand.csv',index_col=0)
data['critical_path']=data['critical_path'].astype('str')

## Regresion models

### Model selection
We use nested cross-validation with the null model (5-rand) simulation dataset for the backward regression problem DBAC~ {activity i's duration at 75%EV} i=1,...,8 

In [4]:
# Reegression models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost.sklearn import XGBRegressor
from interpret.glassbox import ExplainableBoostingRegressor

In [5]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error

In [6]:
# random seed
seed=1123

In [7]:
# DBAC regression
y=data.loc[:,'duration']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [16]:
# outer kfolds
kfold = KFold(n_splits=5, random_state=seed,shuffle=True)

In [10]:
# save outer kfolds
kfs=pd.DataFrame([],columns=['kn','type','i'])
k=0
for train_index, test_index in kfold.split(X):
  p1=pd.DataFrame(train_index,columns=['i'])
  p1['kn']=k
  p1['type']='train'
  p2=pd.DataFrame(test_index,columns=['i'])
  p2['kn']=k
  p2['type']='test'
  p1=pd.concat([p1,p2],ignore_index=True)
  kfs=pd.concat([kfs,p1],ignore_index=True)
  k+=1
kfs.reset_index(inplace=True,drop=True)
kfs.to_csv('./data/regression_model_selection_kf_index.csv')

In [18]:
# load outer kfolds
kfs=pd.read_csv('./data/regression_model_selection_kf_index.csv',index_col=0)

In [24]:
# results
results = pd.DataFrame([['none',0,0]],columns=['model','kf','MSE']) # adding a null row

In [25]:
# ExplainableBoostingRegressor
# https://interpret.ml/docs/python/api/ExplainableBoostingRegressor.html
model='ExplainableBoostingR'
k=0
for train_index, test_index in kfold.split(X):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # inner_bags: number of inner bags.
    # max_leaves: maximum leaf nodes used in boosting.
    # outer_bags: number of outer bags.
  mdr = GridSearchCV(ExplainableBoostingRegressor(),param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                                                                 "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    

  mdr.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results,pd.DataFrame([[model,k, mean_squared_error(y.iloc[test_index],mdr.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','MSE'])],ignore_index=True)
  
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [26]:
results.to_csv('./data/regression_ebm_results.csv',index=False)

In [28]:
# RandomForestRegressor
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
model='RFR'
k=0
for train_index, test_index in kfold.split(X):

  # parameter optimization in the inner folds
    # n_estimators: number of trees
    # max_depth of the trees
  mdr = GridSearchCV(RandomForestRegressor(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'), 
                            "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)  
  mdr.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, 
        mean_squared_error(y.iloc[test_index],mdr.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','MSE'])],ignore_index=True)
  k+=1

In [29]:
results.to_csv('./data/regression_rf_results.csv',index=False)

In [31]:
# GradientBoostingRegressor
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
model='GradientBoostingR'
k=0
for train_index, test_index in kfold.split(X):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of  boosting stages
    # max_depth of the regression estimators
  mdr = GridSearchCV(GradientBoostingRegressor(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),  
                                "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)  
  mdr.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, mean_squared_error(y.iloc[test_index],mdr.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','MSE'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [32]:
results.to_csv('./data/regression_gb_results.csv',index=False)

In [33]:
# AdaBoostRegressor
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html
model='AdaBoostR'
k=0
for train_index, test_index in kfold.split(X):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: maximum number of estimators at which boosting is terminated
  mdr = GridSearchCV(AdaBoostRegressor(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),
                                "learning_rate": np.linspace(0.1,1,5)},
                    n_jobs=6)
  mdr.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, mean_squared_error(y.iloc[test_index],mdr.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','MSE'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [34]:
results.to_csv('./data/regression_ab_results.csv',index=False)

In [35]:
# XGBRegressor
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn
model='XGBoostR'
k=0
for train_index, test_index in kfold.split(X):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of gradient boosted trees
    # max_depth: maximum tree depth for base learners
  mdr = GridSearchCV(XGBRegressor(verbosity = 0),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),  
                                "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)    
  mdr.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, mean_squared_error(y.iloc[test_index],mdr.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','MSE'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [36]:
results.to_csv('./data/regression_xgb_results.csv',index=False)

In [48]:
# Save results
results.drop(results[results.model=='none'].index,inplace=True)
results.reset_index(inplace=True,drop=True)
results.to_csv('./data/regression_model_selection_results.csv')

In [ ]:
# Load results
results=pd.read_csv('./data/regression_model_selection_results.csv',index_col=0)

In [50]:
# Average MSE over 5-fold cross-validation
results.groupby('model').agg({'MSE':['mean','std']}).sort_values(by=('MSE', 'mean'))

MSE          
                           mean       std
model                                    
ExplainableBoostingR   9.165603  0.183922
GradientBoostingR      9.192149  0.197236
RFR                    9.201881  0.170242
XGBoostR               9.209953  0.192083
AdaBoostR             11.078984  0.120717

### Hyper-parameters selection of EBM

#### Forward analysis
DBAC~ {activity i's duration} i=1,...,8

In [51]:
# Regression
y=data.loc[:,'duration']
X=data.loc[:,['duration1','duration2', 'duration3','duration4', 'duration5','duration6', 'duration7','duration8']]

In [52]:
# ExplainableBoostingRegressor
# https://interpret.ml/docs/python/api/ExplainableBoostingRegressor.html
mdr = GridSearchCV(ExplainableBoostingRegressor(),
        param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                    "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    

mdr.fit(X, y)
mdr.best_params_ # 'inner_bags': 6, 'mx_leaves': 2

{'inner_bags': 6, 'max_leaves': 2}

#### Backward analysis
DBAC~ {activity i's duration at 75%EV} i=1,...,8

In [53]:
# Regression
y=data.loc[:,'duration']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [54]:
# ExplainableBoostingRegressor
# https://interpret.ml/docs/python/api/ExplainableBoostingRegressor.html
mdr = GridSearchCV(ExplainableBoostingRegressor(),
        param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                    "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    

mdr.fit(X, y)
mdr.best_params_ # 'inner_bags': 3, 'mx_leaves': 2

{'inner_bags': 3, 'max_leaves': 2}

#### Backward analysis
TB~ {activity i's duration at 75%EV} i=1,...,8

In [55]:
# Regression
y=data.loc[:,'duration@']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [56]:
# ExplainableBoostingRegressor
# https://interpret.ml/docs/python/api/ExplainableBoostingRegressor.html
mdr = GridSearchCV(ExplainableBoostingRegressor(),
        param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                    "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    

mdr.fit(X, y)
mdr.best_params_ # 'inner_bags': 0, 'mx_leaves': 3

{'inner_bags': 0, 'max_leaves': 3}

## Classifier models

In [60]:
# Classifier models
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost.sklearn import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier

In [61]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [62]:
# Classes 
# Expected time of the project 13
data['delay']=data['duration']>13

In [63]:
y=data.loc[:,'delay']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [64]:
# outer kfolds
kfold =StratifiedKFold(n_splits=5, random_state=seed,shuffle=True)

In [66]:
# results
results = pd.DataFrame([['none',0,0]],columns=['model','kf','Accuracy'])

In [67]:
# ExplainableBoostingClassifier
# https://interpret.ml/docs/python/api/ExplainableBoostingClassifier.html
model='ExplainableBoostingC'
k=0
for train_index, test_index in kfold.split(X,y):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of gradient boosted trees
    # max_depth: maximum tree depth for base learners
  mdc = GridSearchCV(ExplainableBoostingClassifier(),
            param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                        "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    
  mdc.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, accuracy_score(y.iloc[test_index],mdc.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','Accuracy'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [68]:
results.to_csv('./data/classification_ebm_results.csv',index=False)

In [71]:
# RandomForestClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
model='RFC'
k=0
for train_index, test_index in kfold.split(X,y):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of trees
    # max_depth of the trees
  mdc = GridSearchCV(RandomForestClassifier(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'), 
                            "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)  
  mdc.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, accuracy_score(y.iloc[test_index],mdc.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','Accuracy'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [72]:
results.to_csv('./data/classification_rf_results.csv',index=False)

In [73]:
# GradientBoostingClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
model='GradientBoostingC'
k=0
for train_index, test_index in kfold.split(X,y):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of  boosting stages
    # max_depth of the regression estimators
  mdc = GridSearchCV(GradientBoostingClassifier(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),  
                                "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)  
  mdc.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, accuracy_score(y.iloc[test_index],mdc.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','Accuracy'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [74]:
results.to_csv('./data/classification_gb_results.csv',index=False)

In [75]:
# AdaBoostClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
model='AdaBoostC'

k=0
for train_index, test_index in kfold.split(X,y):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: maximum number of estimators at which boosting is terminated
  mdc = GridSearchCV(AdaBoostClassifier(),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),
                                "learning_rate": np.linspace(0.1,1,5)},
                    n_jobs=6)
  mdc.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, accuracy_score(y.iloc[test_index],mdc.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','Accuracy'])],ignore_index=True)  
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [76]:
results.to_csv('./data/classification_ab_results.csv',index=False)

In [77]:
# XGBClassifier
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn
model='XGBoostC'
k=0
for train_index, test_index in kfold.split(X,y):
  print('running %i...'%k)
  # parameter optimization in the inner folds
    # n_estimators: number of gradient boosted trees
    # max_depth: maximum tree depth for base learners
  mdc = GridSearchCV(XGBClassifier(verbosity = 0),
                    param_grid={"n_estimators": np.linspace(100,1000,5).astype('int'),  
                                "max_depth": np.linspace(1,10,5).astype('int')},
                    n_jobs=6)    
  mdc.fit(X.iloc[train_index,:], y.iloc[train_index])
  
  # score of the best model in the outer fold
  results = pd.concat([results, pd.DataFrame([[model,k, accuracy_score(y.iloc[test_index],mdc.best_estimator_.predict(X.iloc[test_index,:]))]],columns=['model','kf','Accuracy'])],ignore_index=True)
  k+=1

running 0...
running 1...
running 2...
running 3...
running 4...


In [78]:
results.to_csv('./data/classification_xgb_results.csv',index=False)

In [80]:
# Save results
results.drop(results[results.model=='none'].index,inplace=True)
results.reset_index(inplace=True,drop=True)
results.to_csv('./data/classification_model_selection_results.csv')

In [ ]:
# Load results
results=pd.read_csv('./data/classification_model_selection_results.csv',index_col=0)

In [81]:
# Average Accuracy over 5-fold cross-validation
results.groupby('model').agg({'Accuracy':['mean','std']}).sort_values(by=('Accuracy', 'mean'),ascending=False)

Accuracy          
                         mean       std
model                                  
ExplainableBoostingC  0.85846  0.002716
GradientBoostingC     0.85706  0.003024
RFC                   0.85704  0.001481
XGBoostC              0.85700  0.002124
AdaBoostC             0.85466  0.002503

### Hyper-parameters selection

#### Backward analysis
C_dbac~ {activity i's duration at 75%EV} i=1,...,8

In [82]:
y=data.loc[:,'delay']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [83]:
# ExplainableBoostingClassifier
# https://interpret.ml/docs/python/api/ExplainableBoostingClassifier.html
mdc = GridSearchCV(ExplainableBoostingClassifier(),
            param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                        "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    

mdc.fit(X, y)
mdc.best_params_ # {'inner_bags': 6, 'max_leaves': 2}

{'inner_bags': 6, 'max_leaves': 2}

#### Backward analysis
C_tb~ {activity i's duration at 75%EV} i=1,...,8

In [84]:
# Expected time of the project at 75%EV 9.1763
data['delay@']=data['duration@']>9.1763

In [85]:
y=data.loc[:,'delay@']
X=data.loc[:,['duration@1','duration@2', 'duration@3','duration@4', 'duration@5','duration@6', 'duration@7','duration@8']]

In [86]:
# ExplainableBoostingClassifier
# https://interpret.ml/docs/python/api/ExplainableBoostingClassifier.html
mdc = GridSearchCV(ExplainableBoostingClassifier(),
            param_grid={"inner_bags": np.linspace(0,6,3).astype('int'),
                        "max_leaves": np.linspace(1,3,3).astype('int')},n_jobs=6)    
mdc.fit(X, y)
mdc.best_params_ # {'inner_bags': 6, 'max_leaves': 3}

{'inner_bags': 6, 'max_leaves': 3}